Referance
- 数据处理[https://blog.csdn.net/erinapple/article/details/81174918]
- stacking[https://blog.csdn.net/song430/article/details/90232554]

In [ ]:
#常用工具库
import re
import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt
import time

#算法辅助 & 数据
import sklearn
from sklearn.model_selection import KFold, RepeatedKFold, StratifiedKFold
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import mean_squared_error, roc_auc_score, precision_score, f1_score, recall_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

#算法（单一学习器）
from sklearn.neighbors import KNeighborsClassifier as KNNC
from sklearn.ensemble import AdaBoostClassifier as ABC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import LogisticRegression as LogiR
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import GradientBoostingRegressor as GBR
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb
import catboost as cgb

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

#融合模型
from sklearn.ensemble import StackingClassifier

In [ ]:
for package in [lgb,cgb,sklearn,mlp,np,pd,xgb]:
    print(re.findall("([^']*)",str(package))[2],package.__version__)

In [ ]:
import codecs

features = ["user_id","cid","device_type","fix_month","gender","current_role","last_lat","last_lon","hometown","profession","source_channel","work_city_id","home_city_id","work_lon","home_lat","work_lat","home_lon","age_section","age","version","model","label"]
# print(len(features))
# print(features)
label_file = "data/0403_churn_2.csv"

# with codecs.open(label_file) as f:
#     for line in f.readlines():
#         data = line.strip().split("\t")
#         print(data)
#         break

# create a DataFrame from the dictionary
data = pd.read_csv(label_file, delimiter="\t", names=features).drop("cid", axis=1)

# set the column data types
dtype = {
    'user_id': 'int64',
    'device_type': 'int8',
    'fix_month': 'int8',
    'gender': 'int8',
    'current_role': 'int8',
    'last_lat': 'float32',
    'last_lon': 'float32',
    'hometown': 'category',
    'profession': 'category',
    'source_channel': 'category',
    'work_city_id': 'int8',
    'home_city_id': 'int8',
    'work_lon': 'float32',
    'home_lat': 'float32',
    'work_lat': 'float32',
    'home_lon': 'float32',
    'age_section': 'category',
    'age': 'int8',
    'version': 'category',
    'model': 'category',
    'label': 'int8'
}

# apply the column data types to the DataFrame
df = data.astype(dtype)
print(df.dtypes)

df = df.drop(['hometown'], axis=1)

# selected_rows_0 = df[df['label'] == 0]
# print(len(selected_rows_0))

# selected_rows_1 = df[df['label'] == 1]
# print(len(selected_rows_1))
# df[['user_id','device_type','gender','current_role','source_channel']].head(5)
# df[['user_id','profession','work_city_id','work_lon','work_lat']].sample(5)
# df[['user_id','hometown','home_city_id','home_lat','home_lon']].sample(5)
# df[['user_id','fix_month','version','model']].sample(5)
# df[['user_id','work_city_id','home_city_id']].sample(5)





In [ ]:
# split the data into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# display the sizes of the two subsets
print("Training set size: ", len(train_df))
print("Test set size: ", len(test_df))

train_df.head(2)

X_train = train_df.drop(['user_id', 'label'], axis=1)
y_train = train_df['label']

X_train.head(2)

X_test = test_df.drop(['user_id', 'label'], axis=1)
y_test = test_df['label']


# specify the categorical and numerical columns
categorical_cols = ['hometown', 'profession', 'source_channel', 'age_section', 'version', 'model']

# use one-hot encoding to encode the categorical variables
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_cols)
    ], remainder='passthrough')


In [ ]:
df = preprocessor.fit_transform(pd.concat([X_train,X_test], axis=0))
X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)

In [ ]:
def individual_estimators(estimators):
    """
    对模型融合中每个评估器做交叉验证，对单一评估器的表现进行评估
    """
    print("---individual_estimators-----")
    proba_train = pd.DataFrame()
    proba_test = pd.DataFrame()
    scoring = {'auc': 'roc_auc'}
    for estimator in estimators:
        es_name = estimator[0]
        tic = time.time()
        cv = KFold(n_splits=5,shuffle=True,random_state=1412)
        cv_results = cross_validate(estimator[1],X_train, y_train
                             ,cv = cv
                             ,scoring = scoring
                             ,n_jobs = -1
                             ,return_train_score = True
                             ,verbose=False)

        # print the AUC scores for each fold
        print('Mean Time cost {}s'.format(time.time() - tic))
        print(cv_results[f'test_auc'])

        # calculate the mean AUC score across all folds
        mean_auc = cv_results['test_auc'].mean()
        model = estimator[1].fit(X_train,y_train)
        
        # train_proba = model.predict_proba(X_train)[:, 1]
        train_proba = model.predict_proba(X_train)[:, 1]
        test_proba = model.predict_proba(X_test)[:, 1]
        print('Test Time cost {}s'.format(time.time() - tic))
        
        # calculate the AUC score on the test set
        train_auc = roc_auc_score(y_train, train_proba)
        test_auc = roc_auc_score(y_test, test_proba)

        print(estimator[0]
          ,"\n Mean AUC:{}".format(mean_auc)
          ,"\n train AUC:{}".format(train_auc)
          ,"\n test AUC:{}".format(test_auc)
          ,"\n")
        
        proba_train[es_name] = train_proba
        proba_test[es_name] = test_proba
        
    return proba_train, proba_test
print('---new---')

In [ ]:
def individual_estimators_precision(estimators):
    """
    对模型融合中每个评估器做交叉验证，对单一评估器的表现进行评估
    """
    print("---individual_estimators -----")
    proba_train = pd.DataFrame()
    proba_test = pd.DataFrame()
    # Set the available scoring metrics
    scoring = 'precision'

    # Train a final model on the entire dataset using the same cross-validation object
    final_model_scores = []
    cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=1412)
    for estimator in estimators:
        es_name = estimator[0]
        tic = time.time()
        cv_results = cross_validate(estimator[1],X_train, y_train
                             ,cv = cv
                             ,scoring = scoring
                             ,n_jobs = -1
                             ,return_train_score = True
                             ,verbose=False)

        #dict_keys(['fit_time', 'score_time', 'test_score', 'train_score'])

        # print the AUC scores for each fold
        print('Mean Time cost {}s'.format(time.time() - tic))
        print(cv_results.keys())
        print(cv_results['test_score'])

        # calculate the mean AUC score across all folds
        mean_es = cv_results[f'test_score'].mean()
        model = estimator[1].fit(X_train,y_train)

        y_pred = model.predict(X_test)
        print('Test Time cost {}s'.format(time.time() - tic))
        # calculate the AUC score on the test set
        # train_auc = roc_auc_score(y_train, train_proba)
        test_precision = precision_score(y_test, y_pred, average='macro')
        # print("test AUC:", test_auc)
        print(estimator[0]
          ,"\n Mean AUC:{}".format(mean_es)
          # ,"\n train AUC:{}".format(train_auc)
          ,"\n test AUC:{}".format(test_precision)
          ,"\n")
        
    return proba_train, proba_test
print('---new---')

In [ ]:
# f1
def individual_estimators_f1(estimators):
    """
    对模型融合中每个评估器做交叉验证，对单一评估器的表现进行评估
    """
    print("---individual_estimators precision-----")
    f1_train = pd.DataFrame()
    f1_test = pd.DataFrame()
    # Set the available scoring metrics
    scoring = 'f1'

    # Train a final model on the entire dataset using the same cross-validation object
    cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=1412)
    for estimator in estimators:
        es_name = estimator[0]
        tic = time.time()
        cv_results = cross_validate(estimator[1],X_train, y_train
                             ,cv = cv
                             ,scoring = scoring
                             ,n_jobs = -1
                             ,return_train_score = True
                             ,verbose=False)

        #dict_keys(['fit_time', 'score_time', 'test_score', 'train_score'])

        # print the AUC scores for each fold
        print('Mean Time cost {}s'.format(time.time() - tic))
        print(cv_results.keys())
        print(cv_results['test_score'])

        # calculate the mean AUC score across all folds
        mean_es = cv_results['test_score'].mean()
        model = estimator[1].fit(X_train,y_train)

        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        print('Test Time cost {}s'.format(time.time() - tic))
        # calculate the AUC score on the test set
        train_f1_score = f1_score(y_train, y_train_pred, average='macro')
        test_f1_score = f1_score(y_test, y_test_pred, average='macro')
        # print("test AUC:", test_auc)
        print(estimator[0]
          ,"\n Mean AUC:{}".format(mean_es)
          ,"\n train_f1_score:{}".format(train_f1_score)
          ,"\n test_f1_score:{}".format(test_f1_score)
          ,"\n")
        
        f1_train[es_name] = y_train_pred
        f1_test[es_name] = y_test_pred
        
    return f1_train, f1_test
print('---new---')

In [ ]:
# recall
def individual_estimators_recall(estimators):
    """
    对模型融合中每个评估器做交叉验证，对单一评估器的表现进行评估
    """
    print("---individual_estimators recall-----")
    proba_train = pd.DataFrame()
    proba_test = pd.DataFrame()
    # Set the available scoring metrics
    scoring = 'recall'

    # Train a final model on the entire dataset using the same cross-validation object
    cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=1412)
    for estimator in estimators:
        es_name = estimator[0]
        tic = time.time()
        cv_results = cross_validate(estimator[1],X_train, y_train
                             ,cv = cv
                             ,scoring = scoring
                             ,n_jobs = -1
                             ,return_train_score = True
                             ,verbose=False)

        #dict_keys(['fit_time', 'score_time', 'test_score', 'train_score'])

        # print the AUC scores for each fold
        print('Mean Time cost {}s'.format(time.time() - tic))
        print(cv_results.keys())
        print(cv_results['test_score'])

        # calculate the mean AUC score across all folds
        mean_es = cv_results['test_score'].mean()
        model = estimator[1].fit(X_train,y_train)

        y_pred = model.predict(X_test)
        print('Test Time cost {}s'.format(time.time() - tic))
        # calculate the AUC score on the test set
        # train_auc = roc_auc_score(y_train, train_proba)
        test_recall = recall_score(y_test, y_pred, average='macro')
        # print("test AUC:", test_auc)
        print(estimator[0]
          ,"\n Mean AUC:{}".format(mean_es)
          # ,"\n train AUC:{}".format(train_auc)
          ,"\n test AUC:{}".format(test_recall)
          ,"\n")
        
    return proba_train, proba_test
print('---recall---')

In [ ]:
def fusion_estimators(clf):
    """
    对融合模型做交叉验证，对融合模型的表现进行评估
    """
    tic = time.time()
    scoring = {'auc': 'roc_auc'}
    cv = KFold(n_splits=5,shuffle=True,random_state=42)
    cv_results = cross_validate(clf,X_train,y_train
                             ,cv = cv
                             ,scoring = scoring
                             ,n_jobs = -1
                             ,verbose=True)

    # print the AUC scores for each fold
    print('Mean Time cost {}s'.format(time.time() - tic))
    print(cv_results['test_auc'])

    # calculate the mean AUC score across all folds
    mean_auc = cv_results['test_auc'].mean()
    print("-------------new----------------")

    clf_model = clf.fit(X_train,y_train)
    
    train_proba = clf_model.predict_proba(X_train)[:, 1]
    test_proba = clf_model.predict_proba(X_test)[:, 1]
    print('Test Time cost {}s'.format(time.time() - tic))
    # calculate the AUC score on the test set
    train_auc = roc_auc_score(y_train, train_proba)
    test_auc = roc_auc_score(y_test, test_proba)
    print("fusion estimators:"
          ,"\n Mean AUC:{}".format(mean_auc)
          ,"\n Train AUC:{}".format(train_auc)
          ,"\n Test AUC:{}".format(test_auc)
          ,"\n")

In [ ]:
clf1 = LogiR(max_iter = 500,random_state=42,n_jobs=6)
#增加特征多样性与样本多样性
clf2 = RFC(n_estimators= 100,max_features="sqrt",max_samples=0.9, random_state=42,n_jobs=8)
#特征多样性，稍微上调特征数量
clf3 = GBC(n_estimators= 100,max_features=16,random_state=42) 

#增加算法多样性，新增决策树、KNN、贝叶斯
clf4 = DTC(max_depth=8,random_state=42)
clf5 = KNNC(n_neighbors=10,n_jobs=8)
# clf6 = GaussianNB()

#新增随机多样性，相同的算法更换随机数种子
clf7 = RFC(n_estimators= 100,max_features="sqrt",max_samples=0.9, random_state=4869,n_jobs=8)
clf8 = GBC(n_estimators= 100,max_features=16,random_state=4869)

# Create an SVM classifier with a linear kernel
clf9 = SVC(kernel='poly', degree=3, probability=True)


Logistic Regression
KNN
SVM
GBDT
Decision Tree
RandomForest
AdaBoost
XGBoost
CatBoost
LightGBM

In [ ]:
estimators = [("Logistic Regression",clf1), ("RandomForest", clf2)
              , ("GBDT",clf3), ("Decision Tree", clf4), ("KNN",clf5) 
              , ("RandomForest2", clf7), ("GBDT2", clf8), ("SVM", clf9)
             ]


train_meta, test_meta = individual_estimators(estimators)
final_estimator = RFC(n_estimators=100
                      , min_impurity_decrease=0.0025
                      , random_state= 420, n_jobs=6)
final_estimator.fit(train_meta, y_train)
test_pred = final_estimator.predict_proba(test_meta)[:,1]

# message = 'Test ROC AUC:', roc_auc_score(y_test, test_pred)
# print(message)

In [ ]:
# estimators = [  ("Decision Tree", clf4)
#               , ("RandomForest2", clf7)
#               , ("GBDT2", clf8)
#              ]

# #选择单个评估器中分数最高的随机森林作为元学习器
# #也可以尝试其他更简单的学习器
# final_estimator = RFC(n_estimators=100
#                       , min_impurity_decrease=0.0025
#                       , random_state= 420, n_jobs=6)
# clf = StackingClassifier(estimators=estimators #level0的7个体学习器
#                          ,final_estimator=final_estimator #level 1的元学习器
#                          ,n_jobs=6)
# fusion_estimators(clf)


In [ ]:
base_models = [
    XGBClassifier(learning_rate=0.5,
                    eval_metric='auc',
                    # n_estimators=712,  # 750
                    n_estimators=7,  # 750
                    max_depth=5,
                    min_child_weight=7,
                    gamma=0,
                    subsample=0.8,
                    colsample_bytree=0.6,
                    eta=0.05,
                    silent=1,
                    seed=3,
                    objective='binary:logistic',
                    scale_pos_weight=1),
    LGBMClassifier(num_leaves=35,
                    learning_rate=0.5,
                    # n_estimators=543,  # 443
                    n_estimators=5,  # 443
                    objective='binary',
                    metric='auc',
                    seed=3,
                    colsample_bytree=0.8,
                    min_child_weight=7,
                    subsample=0.8,
                    silent=1),
    CatBoostClassifier(iterations=5,
                        learning_rate=0.5,
                        eval_metric='AUC',
                        depth=8
                        ),
    ABC(n_estimators=50, learning_rate=1)
]

estimators = [("Logistic Regression",clf1), ("KNN",clf5), ("SVM", clf9)
               , ("GBDT2", clf8), ("Decision Tree", clf4), ("RandomForest2", clf7)
              , ("AdaBoost", base_models[3]),("XGBClassifier",base_models[0])
             ]

train_meta, test_meta = individual_estimators(estimators)

In [ ]:
estimators = [("Logistic Regression",clf1), ("KNN",clf5)
               , ("GBDT2", clf8), ("Decision Tree", clf4), ("RandomForest2", clf7)
              , ("AdaBoost", base_models[3]),("XGBClassifier",base_models[0]), ("CatBoostClassifier",base_models[2])
              , ("LGBMClassifier",base_models[1])
             ]

train_meta, test_meta = individual_estimators_recall(estimators)

In [ ]:
# estimators = [("Logistic Regression",clf1), ("KNN",clf5), ("SVM", clf9)
#                , ("GBDT2", clf8), ("Decision Tree", clf4), ("RandomForest2", clf7)
#               , ("AdaBoost", base_models[3]),("XGBClassifier",base_models[0]), ("CatBoostClassifier",base_models[2])
#               , ("LGBMClassifier",base_models[1])
#              ]

estimators = [ ("Logistic Regression",clf1)  
              ,("Decision Tree", clf4)
              , ("RandomForest2", clf7)
            , ("XGBClassifier",base_models[0])
            , ("LGBMClassifier",base_models[1])
             ]

train_meta, test_meta = individual_estimators(estimators)
final_estimator = RFC(n_estimators=100
                      , min_impurity_decrease=0.0025
                      , random_state= 420, n_jobs=6)
final_estimator.fit(train_meta, y_train)
test_pred = final_estimator.predict_proba(test_meta)[:,1]

message = 'Test ROC AUC:', roc_auc_score(y_test, test_pred)
print(message)


In [ ]:
train_meta, test_meta = individual_estimators_f1(estimators)
final_estimator = RFC(n_estimators=100
                      , min_impurity_decrease=0.0025
                      , random_state= 420, n_jobs=6)
final_estimator.fit(train_meta, y_train)
test_pred = final_estimator.predict(test_meta)

message = 'Test F1:', f1_score(y_test, test_pred)
print(message)


In [ ]:
estimators = [("XGBClassifier",base_models[0])
              ,("LGBMClassifier",base_models[1])
              , ("Decision Tree", clf4)
              , ("RandomForest2", clf7)
              , ("GBDT2", clf8)
             ]

#选择单个评估器中分数最高的随机森林作为元学习器
#也可以尝试其他更简单的学习器
final_estimator = RFC(n_estimators=100
                      , min_impurity_decrease=0.0025
                      , random_state= 420, n_jobs=-1)
clf = StackingClassifier(estimators=estimators #level0的7个体学习器
                         ,final_estimator=final_estimator #level 1的元学习器
                         ,n_jobs=-1)
fusion_estimators(clf)

In [ ]:


label_file = "data/0403_churn_2.csv"
# create a DataFrame from the dictionary
data = pd.read_csv(label_file, delimiter="\t", names=["user_id","cid","device_type","fix_month","gender","current_role","last_lat","last_lon","hometown","profession","source_channel","work_city_id","home_city_id","work_lon","home_lat","work_lat","home_lon","age_section","age","version","model","label"]).drop("cid", axis=1)

# one-hot encode the categorical features
cat_features = ['hometown', 'profession', 'source_channel', 'age_section', 'version', 'model']
df = pd.get_dummies(data, columns=cat_features)

# set the column data types
dtype = {
    'user_id': 'int64',
    'device_type': 'int8',
    'fix_month': 'int8',
    'gender': 'int8',
    'current_role': 'int8',
    'last_lat': 'float32',
    'last_lon': 'float32',
    'work_city_id': 'int32',
    'home_city_id': 'int32',
    'work_lon': 'float32',
    'home_lat': 'float32',
    'work_lat': 'float32',
    'home_lon': 'float32',
    'age': 'int8',
    'label': 'int8'
}

# apply the column data types to the DataFrame
df = df.astype(dtype)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
X_train = train_df.drop('label', axis=1)
y_train = train_df['label']
X_test = test_df.drop('label', axis=1)
y_test = test_df['label']
data_message = 'X_train.shape={}, X_test.shape={}'.format(X_train.shape, X_test.shape)

In [ ]:
print(df.shape)
print(len(df))

In [ ]:
label_file = "data/0403_churn_2.csv"
data = pd.read_csv(label_file, delimiter="\t", names=["user_id","cid","device_type","fix_month","gender","current_role","last_lat","last_lon","hometown","profession","source_channel","work_city_id","home_city_id","work_lon","home_lat","work_lat","home_lon","age_section","age","version","model","label"]).drop("cid", axis=1)

# set the column data types
dtype = {
    'user_id': 'int64',
    'device_type': 'int8',
    'fix_month': 'int8',
    'gender': 'int8',
    'current_role': 'int8',
    'last_lat': 'float32',
    'last_lon': 'float32',
    'hometown': 'category',
    'profession': 'category',
    'source_channel': 'category',
    'work_city_id': 'int8',
    'home_city_id': 'int8',
    'work_lon': 'float32',
    'home_lat': 'float32',
    'work_lat': 'float32',
    'home_lon': 'float32',
    'age_section': 'category',
    'age': 'int8',
    'version': 'category',
    'model': 'category',
    'label': 'int8'
}

# apply the column data types to the DataFrame
df = data.astype(dtype)
df['hometown'] = df['hometown'].cat.add_categories('NA')
df['hometown'] = df['hometown'].fillna('NA')

if type(df) == pd.DataFrame:
    df = df.values
print(type(df))

In [ ]:
params = {
            'learning_rate': 0.05,
            'eval_metric': 'AUC',
            'depth': 8,
            'logging_level': 'Info',
            'loss_function': 'Logloss',
            'train_dir': 'model/cgb_record/',
            'thread_count': 6
        }
max_round = 1
cv_folds = 2
seed = 3
save_model_path = 'model/cgb.model'

dtrain = cgb.Pool(X_train, label=y_train)
cv_result = cgb.cv(dtrain, params, num_boost_round=max_round, nfold=cv_folds, seed=seed, logging_level='Verbose')

print(cv_result.keys())
for key, value in cv_result.items():
    print(key, value)

auc_test_mean = cv_result['test-AUC-mean']
best_round = np.argmax(auc_test_mean)
best_auc = np.max(auc_test_mean)  # 最好的 auc 值
